In [1]:
import json
import base64
import mimetypes
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

c:\Users\ansha\Downloads\split-bill-app-main\Assignment_Rifqi Anshari Rasyid\.venv\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.4) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


True

In [12]:
client = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", 
    temperature=0,
    max_retries=0
)

In [3]:
IMAGE_PATH = r"../data/receipt_3.jpg"
PROMPT = """
You are given an image of a receipt. Please read the content into JSON format:

```
{
    "items": [
        {
            "name": <item name>,
            "quantity": <item quantity>,
            "price_per_unit": <item price per unit>
        }
    ],
    "service_price": <service price in receipt or 0 if not available>,
    "tax_price": <tax price in receipt or 0 if not available>,
    "discount_price": <discount price in receipt or 0 if not available>,
}
```

return only in JSON format
"""

In [4]:
with open(IMAGE_PATH, "rb") as f:
    image_bytes = f.read()

mime_type, _ = mimetypes.guess_type(IMAGE_PATH)
encoded = base64.b64encode(image_bytes).decode("utf-8")
image_uri = f"data:{mime_type};base64,{encoded}"

In [5]:
message = HumanMessage(
    content=[
        {"type": "text", "text": PROMPT},
        {
            "type": "image_url",
            "image_url": {"url": image_uri},
        },
    ]
)
response = client.invoke([message])
print(response.content)

```json
{
    "items": [
        {
            "name": "POP MIE AYAM 75G",
            "quantity": 1,
            "price_per_unit": 4900
        },
        {
            "name": "POP MIE PD.DWR AYM75",
            "quantity": 1,
            "price_per_unit": 5400
        },
        {
            "name": "NESTLE PURE LIFE 600",
            "quantity": 2,
            "price_per_unit": 3600
        },
        {
            "name": "LE MINERALE 600ML",
            "quantity": 2,
            "price_per_unit": 3500
        },
        {
            "name": "ULTRA KCNG HIJAU 250",
            "quantity": 1,
            "price_per_unit": 4900
        },
        {
            "name": "NUTRIJEL PWD.STRW.15",
            "quantity": 2,
            "price_per_unit": 6600
        },
        {
            "name": "KNZLER SNGLES KJU 65",
            "quantity": 1,
            "price_per_unit": 8700
        },
        {
            "name": "KNZLER SNGLES HOT 65",
            "quantity": 1,
            

In [6]:
json_string = response.content.replace("```json", "").replace("```", "")
receipt_dict = json.loads(json_string)
receipt_dict

{'items': [{'name': 'POP MIE AYAM 75G', 'quantity': 1, 'price_per_unit': 4900},
  {'name': 'POP MIE PD.DWR AYM75', 'quantity': 1, 'price_per_unit': 5400},
  {'name': 'NESTLE PURE LIFE 600', 'quantity': 2, 'price_per_unit': 3600},
  {'name': 'LE MINERALE 600ML', 'quantity': 2, 'price_per_unit': 3500},
  {'name': 'ULTRA KCNG HIJAU 250', 'quantity': 1, 'price_per_unit': 4900},
  {'name': 'NUTRIJEL PWD.STRW.15', 'quantity': 2, 'price_per_unit': 6600},
  {'name': 'KNZLER SNGLES KJU 65', 'quantity': 1, 'price_per_unit': 8700},
  {'name': 'KNZLER SNGLES HOT 65', 'quantity': 1, 'price_per_unit': 8700},
  {'name': 'KANZLR BAKSO ORI 48G', 'quantity': 2, 'price_per_unit': 8700}],
 'service_price': 0,
 'tax_price': 7036,
 'discount_price': 2000}

In [7]:
# for name in receipt_dict["menus"]:
#     print(f"Item: {name['name']}, Count: {name['count']}, Price: {name['price']}")

In [8]:
def split_evenly(receipt_dict, num_people):
    sub_total = sum(item["price_per_unit"] * item["quantity"] for item in receipt_dict["items"])
    total_price = sub_total + receipt_dict.get("service_price", 0) + receipt_dict.get("tax_price", 0) - receipt_dict.get("discount_price", 0)
    return round(total_price / num_people)

num_person = 3
split_evenly(receipt_dict, num_person)

27479

In [11]:
def split_by_items(receipt_dict, assignments):
    sub_total = sum(item["price_per_unit"] * item["quantity"] for item in receipt_dict["items"])
    items_dict = {item["name"]: {"quantity": item["quantity"], "price": item["price_per_unit"]} for item in receipt_dict["items"]}

    result = {}
    for user, items in assignments.items():
        sub_total_user = 0
        for name, quantity in items.items():
            if name not in items_dict:
                raise ValueError(f"Item '{name}' not found.")
            if quantity <= 0:
                raise ValueError("Item quantity must be positive.")
            if quantity > items_dict[name]["quantity"]:
                raise ValueError(f"Quantity exceeds available quantity for '{name}'.")

            sub_total_user += items_dict[name]["price_per_unit"] * quantity

        proportion = sub_total_user / sub_total
        result[user] = round(sub_total_user + proportion * (receipt_dict.get("service_price", 0) + receipt_dict.get("tax_price", 0) - receipt_dict.get("discount_price", 0)))

    return result

assignments = {
    "Budi": {"Bintang Bremer": 1, "Chicken H-H": 1},
    "Rina": {"Ades": 1}
}
split_by_items(receipt_dict, assignments)

ValueError: Item 'Bintang Bremer' not found.